# Startup for Canada
This code is to be included at the beginning of `calib_CAN` and `simu_CAN`.

In [1]:
t0 = time() ## start chrono
dir_module = joinpath(pwd(),"src")
dir_graph = joinpath(pwd(),"../../graphs")
    
########### main module
using NBInclude
@nbinclude(joinpath(dir_module,"AgeDisc.ipynb"))
using Main.AgeDisc

########### other packages
using QuadGK, DataFrames, PyPlot

### Some variables can be predefined before startup.jl : _PRECOMPILE_, _LC_
## _PRECOMPILE_ is whether to run a few functions so that they are already compiled
if !@isdefined(_PRECOMPILE_)
    _PRECOMPILE_ = true
end

if !@isdefined(_LC_)
    _LC_ = false
end

false

# Moments

In [2]:
data_can = DataFrame([
(25, 0.677),
(26, 0.655),
(27, 0.669),
(28, 0.663),
(29, 0.662),
(30, 0.609),
(31, 0.597),
(32, 0.573),
(33, 0.596),
(34, 0.556),
(35, 0.577),
(36, 0.545),
(37, 0.535),
(38, 0.548),
(39, 0.522)], [:age, :rate])
data_can.age *=12

moments_target, Age_vec = let
    Age_bds = (27,35) .*12 ## window to match the moments
    myselec = Age_bds[1] .<= data_can.age .<= Age_bds[2]
    (data_can.rate[myselec], data_can.age[myselec])
end

function moments_canada(ureal; 
        atol=1e-8, a_range=Age_vec)
    Y_vec = [quadgk(x->1-sum(ureal(x)[:]) ,a, a+12, atol=atol)[1]/12 for a in a_range]
    return Y_vec
end


function UIpolicy(jobspell) ## from Canadian Statistics in 1986, Lin (1998)
    ## caveat: we only account for continuous employment spell
    ## jobspell is in months
    if jobspell < 20 * 12/52 ## no eligible below 20 weeks
        UImax = 0.
    else ## eligible
        if jobspell < 25 *12/52
            UImax = jobspell
        else
            UImax = 25. *12/52 + min( (jobspell-25*12/52) /2, 13 * 12/52)
        end
        UImax += min(32, (10-4)/0.5 * 2) * 12/52 ## 24 is regional component with 10% unemployment rate
        UImax = min(50*12/52 , UImax)
    end
    return(UImax)
end

lambda_function(q) = 1/quadgk(x-> UIpolicy(x) * q * exp(-q*x), 0, Inf)[1] 

lambda_function (generic function with 1 method)

# Parameters

In [3]:
if !_LC_
    C0 = Country1(
            A = 30. * 12, Amin = 25. *12, Amax = 40. *12,
            r = 0.0048, # 5 % annual interest rate, time unit = month
            α = NaN, m = NaN, q = NaN, λ = NaN,
            uinit = [0.32,0.], ## start with 68% employment rate, 32% without UI, 0% with UI
            b1 = 507 * 12/13924, # 507 and 13924 (average earnings) in 1989 (current dollars), from LM2008
            b0 = 185 * 12/13924, # 185 from LM2008
            bUI = 0.6, ## from Canadian statistics
            B1=NaN, B2=NaN, B1dif=NaN
    )    

    function update_Country1(;α=C0.α, m=C0.m, q=C0.q)
        (B1, B1dif, B2) = Baux(α=α, m=m)
        Country1(A=C0.A, Amin=C0.Amin, Amax=C0.Amax,
                        r=C0.r, q=q, m=m, λ=lambda_function(q), α=α,
                        bUI=C0.bUI, b0=C0.b0, b1=C0.b1,
                        uinit=C0.uinit,
                        B1=B1, B1dif=B1dif, B2=B2)
    end
    
    
else
    C0 = Country1A(
            A = 30. * 12, Amin = 18. *12, Amax = 40. *12, Aret = 65. * 12, ## no mandatory retirement age
            r = 0.0048, # 5 % annual interest rate, time unit = month
            α = NaN, m = NaN, q0 = NaN, qa=NaN, λ = NaN, ξ=0.,
            uinit = [1.,0.], #[0.32,0.], ## start with 0% employment rate at Amin
            b1 = 507 * 12/13924, # 507 and 13924 (average earnings) in 1989 (current dollars), from LM2008
            b0 = 185 * 12/13924, # 185 from LM2008
            bUI = 0.6, ## from Canadian statistics
            B1=NaN, B2=NaN, B1dif=NaN
    )    

    function update_Country1(;α=C0.α, m=C0.m, q0=C0.q0, qa=C0.qa, ξ=C0.ξ, Aret=C0.Aret)
        (B1, B1dif, B2) = Baux(α=α, m=m)
        Country1A(A=C0.A, Amin=C0.Amin, Amax=C0.Amax, Aret=Aret,
                        r=C0.r, q0=q0, qa=qa, m=m, λ=lambda_function(q0), α=α, ξ=ξ,
                        bUI=C0.bUI, b0=C0.b0, b1=C0.b1,
                        uinit=C0.uinit,
                        B1=B1, B1dif=B1dif, B2=B2)
    end
end

update_Country1 (generic function with 1 method)

In [4]:
println("C0 is $C0")
t1 = time()
nsec = (t1-t0)
println("Startup run in $nsec secs.")

C0 is Country1(360.0, 300.0, 480.0, 0.0048, NaN, NaN, NaN, NaN, [0.32, 0.0], 0.6, 0.4369434070669348, 0.15943694340706693, NaN, NaN, NaN)
Startup run in 22.375999927520752 secs.
